### Types of Recomendation system

1. Popularity(Formula Based)
2. Content Based (Tags like comedy , thriller, love, etc,.)
3. Collaborative filtering (Based on users similarity)
4. Hybrid (Collection of all the above 3)

We are going to develop using Content Based

In [35]:
import pandas as pd

In [36]:
music_df = pd.read_csv("spotify_millsongdata.csv")

In [37]:
music_df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [38]:
music_df.shape

(57650, 4)

In [39]:
music_df.columns

Index(['artist', 'song', 'link', 'text'], dtype='object')

In [40]:
music_df.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


In [41]:
music_df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [42]:
# Drop the link column and take a small sample 5000 from the big dataset and reset the index
df = music_df.sample(5000).drop('link',axis =1).reset_index(drop=True)

In [43]:
df.shape

(5000, 3)

In [44]:
df['text'][1]

"I was a young man when I died  \r\nI was a flash, I was full of pride  \r\nI gave it all, I gave my soul, I was so strong  \r\nI felt the truth, I felt the pain in every song  \r\n  \r\nHow well you know me  \r\nYou've seen me cry  \r\nI'm just a shadow  \r\nIn a rock and roll sky  \r\n  \r\nI was living in a rock and roll sky  \r\nI was a king, heavy on the style  \r\nSuper trouper, yes I know you well, making me shine  \r\nI couldn't see what you did to me.  \r\nI was so blind  \r\n  \r\nHow well you know me  \r\nYou've seen me cry  \r\nI'm just a shadow  \r\nIn a rock and roll sky  \r\n  \r\nI wanna be like I was before  \r\nBut this time I'm gonna know the score  \r\nI need a home I can leave behind knowing you're there  \r\nSuper trouper, I can see you now, return your stare  \r\n  \r\nHow well you know me  \r\nYou've seen me cry  \r\nI'm just a shadow  \r\nIn a rock and roll sky\r\n\r\n"

In [45]:
df.columns

Index(['artist', 'song', 'text'], dtype='object')

### Text Preprocessing

In [46]:
# COnvert the text into lower case, replace all the characters except alphabet-digits[\w] and white space[\s] to empty string
# COnvert \n with ' '
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [47]:
df['text']

0       dancin' in the dark 'til the tune ends  \r we'...
1       i was a young man when i died  \r i was a flas...
2       time, is going by, so much faster than i,  \r ...
3       bad times, something's wrong  \r money's gone,...
4       i saw an open door and i looked inside it  \r ...
                              ...                        
4995    chorus:  \r i know who i am  \r i know who i a...
4996    domo arigato misuta robotto  \r mata au hi mad...
4997    saw you layin' under my car, but you took it t...
4998    oh since my baby left me i've found new place ...
4999    it was a hundred and seven  \r we were heading...
Name: text, Length: 5000, dtype: object

In [48]:
df.tail()

,artist,song,text
4995,Israel,I Know Who I Am,chorus: \r i know who i am \r i know who i a...
4996,Styx,Mr. Roboto,domo arigato misuta robotto \r mata au hi mad...
4997,Mazzy Star,Under My Car,"saw you layin' under my car, but you took it t..."
4998,Willie Nelson,Heartbreak Hotel,oh since my baby left me i've found new place ...
4999,Garth Brooks,That Girl Is A Cowboy,it was a hundred and seven \r we were heading...


## Tokenization

In [49]:
import nltk
from nltk.stem.porter import PorterStemmer

In [50]:
stemmer = PorterStemmer()

In [54]:
# Function to convert all text to token with stem function and converting to each word to it's root word
def token(txt):
    token = nltk.word_tokenize(txt)
    base_word = [stemmer.stem(word) for word in token]
    return ' '.join(base_word)

In [55]:
# Test for function with example
token("Recomend, Recommendation, Recomending")

'recomend , recommend , recomend'

In [56]:
# Apply the token function to dataset 'text' column
df['text'].apply(lambda x: token(x))

0       dancin ' in the dark 'til the tune end we 're ...
1       i wa a young man when i die i wa a flash , i w...
2       time , is go by , so much faster than i , and ...
3       bad time , someth 's wrong money 's gone , i '...
4       i saw an open door and i look insid it but i '...
                              ...                        
4995    choru : i know who i am i know who i am i know...
4996    domo arigato misuta robotto mata au hi made do...
4997    saw you layin ' under my car , but you took it...
4998    oh sinc my babi left me i 've found new place ...
4999    it wa a hundr and seven we were head to town s...
Name: text, Length: 5000, dtype: object

### Vectorization

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [57]:
# tf: term frequency = word count in the document/ total words in sentence 
# idf: Inverse Document frequency =  Total document/ Word count in doucumnet
# tfidf = Tf* Idf
tfidV = TfidfVectorizer(analyzer='word', stop_words='english')

In [58]:
vector = tfidV.fit_transform(df['text'])

In [59]:
# Apply the cosine similarity
# COnsine similarity get the Angle between the two vetors
similar = cosine_similarity(vector)

In [60]:
similar.shape

(5000, 5000)

In [24]:
similar

array([[1.        , 0.04576768, 0.06215841, ..., 0.0078641 , 0.00212612,
        0.0627998 ],
       [0.04576768, 1.        , 0.04470051, ..., 0.00748024, 0.        ,
        0.07955186],
       [0.06215841, 0.04470051, 1.        , ..., 0.00869395, 0.0185206 ,
        0.05887047],
       ...,
       [0.0078641 , 0.00748024, 0.00869395, ..., 1.        , 0.        ,
        0.05838081],
       [0.00212612, 0.        , 0.0185206 , ..., 0.        , 1.        ,
        0.02069967],
       [0.0627998 , 0.07955186, 0.05887047, ..., 0.05838081, 0.02069967,
        1.        ]])

## Recommendation System

In [62]:
df[df['song']=='I Know Who I Am'].index[0]

4995

In [63]:
df[df['song']=='I Know Who I Am']

,artist,song,text
4995,Israel,I Know Who I Am,chorus: \r i know who i am \r i know who i a...


In [30]:
# recommender function

def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    
    # Get the whole row of similar matrix corresponse to the index and Sort it in reverse order based on vector value
    # Also get the index value using enumerate function
    vector_idx = sorted(list(enumerate(similar[idx])), reverse = True, key = lambda x:x[1])
    songs = []
    
    # Make use of Index value to get the song names of top 10 songs, 0th index is sikped as that is the song itself
    for s_id in vector_idx[1:11]:
        songs.append(df.iloc[s_id[0]].song)
    return songs

In [65]:
# Function test
recommender("I Know Who I Am")

['Be Still',
 'You Belong With Me',
 'Just Like Them',
 'Everything Has Changed',
 'You Belong To Me',
 'The Song Is Alive',
 'My Soul Is Complete',
 'Belong To The World',
 'Back To What You Know',
 'Everything To Me']

## Store the model created

In [66]:
import pickle

In [67]:
# dump the model in a file with write mode
pickle.dump(similar,open("recommendation.pkl","wb"))

In [68]:
# dump the dataset in the file
pickle.dump(df,open("recommendation_df.pkl","wb"))